In [ ]:
import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

In [ ]:
filename = 'ntuple_test.root'
f = uproot.open(filename)

In [ ]:
f.keys()

In [ ]:
tree = f['muonPhiAnalyzer/tree']

tree.keys()

In [ ]:
x = tree['muon_dtSeg_globX'].array()
y = tree['muon_dtSeg_globY'].array()
z = tree['muon_dtSeg_globZ'].array()
t = tree['muon_dtSeg_t0timing'].array()

In [ ]:
x

In [ ]:
type(ak.to_numpy(z[0]))

In [ ]:
for i,xd in enumerate(ak.num(x)):
    print(i,xd)

In [ ]:
tree.num_entries

https://matplotlib.org/stable/api/toolkits/mplot3d/view_angles.html

In [ ]:
################################################################################
# Takes in a starter plane and then rotates it around the provided angles
################################################################################
def generate_muon_detector_planes(x,y,z):
    planes = []
    angles = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
    for angle in angles:
        theta = np.deg2rad(angle)
        #rot = np.array([[np.cos(theta),-np.sin(theta)], [np.sin(theta),np.cos(theta)]])
        # Rotate around x-axis
        rot = np.array([[1,0,0],[0,np.cos(theta),-np.sin(theta)], [0,np.sin(theta),np.cos(theta)]])

        #print(x.shape,y.shape,z.shape)
        #print(x)
        #print(y)
        #print(z)

        x1 = np.ones(shape=x.shape)
        y1 = np.ones(shape=y.shape)
        z1 = np.ones(shape=z.shape)

        imax = x.shape[0]
        jmax = x.shape[1]
        for i in range(imax):
            for j in range(jmax):
                v = [x[i][j], y[i][j], z[i][j]]
                v1 = np.matmul(rot,v)
                #print("here: ",v1)
                x1[i][j] = v1[0]
                y1[i][j] = v1[1]
                z1[i][j] = v1[2]

        planes.append([x1,y1,z1])

    return planes
##############################################################################

In [ ]:
#####################################################
# Get points
#####################################################
xdata = tree['muon_dtSeg_globX'].array()
ydata = tree['muon_dtSeg_globY'].array()
zdata = tree['muon_dtSeg_globZ'].array()
tdata = tree['muon_dtSeg_t0timing'].array()

#n = 1
n = 64
n = 93
n = 2101
n = 2352
n = 2604
n = 2607

#x[0]

xpts = ak.to_numpy(xdata[n])
ypts = ak.to_numpy(ydata[n])
zpts = ak.to_numpy(zdata[n])
tpts = ak.to_numpy(tdata[n])

mask = np.abs(tpts) < 100

xpts = xpts[mask]
ypts = ypts[mask]
zpts = zpts[mask]
tpts = tpts[mask]



fig = plt.figure(figsize=(20,60), num=1, clear=True)
axes = []
'''
axes.append(fig.add_subplot(2, 2, 1, projection='3d'))
axes.append(fig.add_subplot(2, 2, 2, projection='3d'))
axes.append(fig.add_subplot(2, 2, 3, projection='3d'))
axes.append(fig.add_subplot(2, 2, 4, projection='3d'))
'''
axes.append(fig.add_subplot(4, 1, 1, projection='3d'))
axes.append(fig.add_subplot(4, 1, 2, projection='3d'))
axes.append(fig.add_subplot(4, 1, 3, projection='3d'))
axes.append(fig.add_subplot(4, 1, 4, projection='3d'))

'''
fig2 = plt.figure(figsize=(12,8), num=2, clear=True)
axes2 = []
axes2.append(fig2.add_subplot(2, 2, 1, projection='3d'))
axes2.append(fig2.add_subplot(2, 2, 2, projection='3d'))
axes2.append(fig2.add_subplot(2, 2, 3, projection='3d'))
axes2.append(fig2.add_subplot(2, 2, 4, projection='3d'))
'''


planes = []
planes_for_display = []

# https://twiki.cern.ch/twiki/bin/view/CMSPublic/MuonDPGPublic160729#Spatial_Resolution
#radii = [4.0, 5.0, 6.0, 7.0] # Meters
#radii = [4.0, 5.0, ] # Meters
radii = [4.0,4.05,4.10,4.15, 5.0,5.05,5.10,5.15, 6.0,6.05,6.10,6.15, 7.0,7.05,7.10,7.15] # Meters
#radii = [4.0,4.05,4.10,4.15,]
#radii = [4.0] # Meters
length = 13.0/2 # Meters
for radius in radii:

    width = np.deg2rad(30)*radius/2
    #print(radius, "\twidth: ",width)

    # The real planes.
    #(x, y) = np.meshgrid(np.arange(-length, length+0.0001, 0.05), np.arange(-width, width+0.0001, .02))
    (x, y) = np.meshgrid(np.arange(-length, length+0.0001, 0.5), np.arange(-width, width+0.0001, .1))
    z = radius*np.ones(shape=x.shape)
    planes += generate_muon_detector_planes(x,y,z)

    # Planes just for display purposes with a much coarser binning.
    (x, y) = np.meshgrid(np.arange(-length, length+0.0001, 2.0), np.arange(-width, width+0.0001, .5))
    z = radius*np.ones(shape=x.shape)
    planes_for_display += generate_muon_detector_planes(x,y,z)

for p in planes_for_display:
    x = p[0]
    y = p[1]
    z = p[2]
    for ax in axes:
        ax.plot_wireframe(x, y, z,color='tan',alpha=0.10)
        #ax.plot_surface(x, y, z,color='tan',alpha=0.1)
        ax.plot(xpts/100, ypts/100, zpts/100,'o',markersize=10, alpha=0.1)


#####################################################

axes[1].azim += 30
axes[1].elev -= 30

axes[2].azim = 90

axes[3].azim = 0
axes[3].elev = 0

#plt.tight_layout()

print(tpts)
print(xpts)
print(ypts)
print(zpts)

In [ ]:
n = 2063

#x[0]

xpts = ak.to_numpy(x[n])
ypts = ak.to_numpy(y[n])
zpts = ak.to_numpy(z[n])
tpts = ak.to_numpy(t[n])


fig = plt.figure(figsize=(15,15))

for i in range(0,4):

    ax = fig.add_subplot(2,2,i+1, projection='3d')
    ax.plot(xpts, ypts, zpts,'o',markersize=10, alpha=0.3)

    ax.set_xlim(-800, 800)
    ax.set_ylim(-800, 800)
    ax.set_zlim(-1200, 1200)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    if i==0:
        ax.view_init(elev=10., azim=10)
    elif i==1:
        ax.view_init(elev=90., azim=-90, roll=0)
    elif i==2:
        ax.view_init(elev=0., azim=180, roll=0)

plt.tight_layout()

print(t[n])

In [ ]:
ak.num(x)

In [ ]:
plt.hist(ak.flatten(t),bins=100,range=(-80,80));

In [ ]:
ccs = []

for i in range(len(x)):
    cc = np.corrcoef(y[i],t[i])
    #print(x[i],cc[1][0])
    ccs.append(cc[1][0])

In [ ]:
plt.hist(ccs,bins=25);

In [ ]:
x.array()

In [ ]:
for a,b,c in zip(t[1], x[1],y[1]):
    print(a,b,c)


In [ ]:
r = (x**2 + y**2 + z**2)**(0.5)
r

In [ ]:
plt.plot(ak.flatten(t),ak.flatten(r),'o')
plt.xlim(-1200,1000)
plt.ylim(0,1000)

In [ ]:
plt.plot(ak.flatten(t),ak.flatten(r),'.',alpha=0.05, markersize=1)
plt.xlim(-250,250)
plt.ylim(400,1000)